In [16]:
import pandas as pd 
import numpy as np
import dill

from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

from sklearn.metrics import mean_absolute_error

from dython.data_utils import identify_columns_with_na

In [2]:
X_train_FE_encode = pd.read_csv('dataset/train_data_FE.csv')

In [3]:
X_test_FE_encode = pd.read_csv('dataset/test_data_FE.csv')

In [4]:
X_train_FE_encode["Sale_Price"].to_csv('dataset/y_train.csv', index=False)
X_test_FE_encode["Sale_Price"].to_csv('dataset/y_test.csv', index=False)

In [5]:
y_train= pd.read_csv('dataset/y_train.csv').squeeze() #converte in Series
y_test= pd.read_csv('dataset/y_test.csv').squeeze() #converte in Series

# ENCODING E OUTLIERS
- il primo step è quello di fare ordinal encoding sulle features categoriche, facendo riferimento alla descrizione delle feature riportate nel file "description.txt"
- il secondo step prevede oneHot encoding delle altre features categoriche
- il terzo step prevede l'individuazione e la rimozione degli outliers, attraverso una isolation forest(applicata su tutte le features)

# ENCODING CATEGORICAL FEATURES

## Label Encoding

BASEMENT

In [6]:
def encode_basement(df):
    
    mp = {'Excellent':5,'Good':4,'Typical':3,'Fair':2,'Poor':1,'No_Basement':0}
    df['Bsmt_Qual'] = df['Bsmt_Qual'].map(mp)
    df['Bsmt_Cond'] = df['Bsmt_Cond'].map(mp)
    df['Bsmt_Exposure'] = df['Bsmt_Exposure'].map(
        {'Gd':4,'Av':3,'Mn':2,'No':1,'No_Basement':0})

    mp = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No_Basement':0}
    df['BsmtFin_Type_1'] = df['BsmtFin_Type_1'].map(mp)
    df['BsmtFin_Type_2'] = df['BsmtFin_Type_2'].map(mp)
    return df
    

In [7]:
X_train_FE_encode=encode_basement(X_train_FE_encode)


In [8]:
X_test_FE_encode=encode_basement(X_test_FE_encode)

ELECTRICAL

In [9]:
X_train_FE_encode['Electrical'].value_counts()

SBrkr      2013
FuseA       138
FuseF        38
FuseP         6
Unknown       1
Mix           1
Name: Electrical, dtype: int64

In [10]:
def encode_electrical(df):
    df['Electrical'] = df['Electrical'].map(
        {'SBrkr':5,'FuseA':4,'FuseF':3,'FuseP':2,'Mix':1,'Unknown':0})
    return df

In [11]:
X_train_FE_encode=encode_electrical(X_train_FE_encode)
#test

In [12]:
X_test_FE_encode=encode_electrical(X_test_FE_encode)

UTILITIES

In [13]:
X_train_FE_encode['Utilities'].value_counts()

AllPub    2194
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64

In [14]:
def encode_utilities(df):
    df['Utilities'] = df['Utilities'].map({
        'AllPub':3,'NoSewr':2,'NoSeWa':1,'ELO':0})
    return df

In [15]:
X_train_FE_encode=encode_utilities(X_train_FE_encode)
#test


In [16]:
X_test_FE_encode=encode_utilities(X_test_FE_encode)

EXTERIOR AND KITCHEN QUALITY

In [17]:
def encode_exterior_kitchen(df):
    mp = {'Excellent':4,'Good':3,'Typical':2,'Fair':1,'Poor':0}
    df['Exter_Qual'] = df['Exter_Qual'].map(mp)
    df['Exter_Cond'] = df['Exter_Cond'].map(mp)
    df['Kitchen_Qual'] = df['Kitchen_Qual'].map(mp)
    return df

In [18]:
X_train_FE_encode=encode_exterior_kitchen(X_train_FE_encode)
#test

In [19]:
X_test_FE_encode=encode_exterior_kitchen(X_test_FE_encode)

FIREPALCE

In [20]:
def encode_fireplace(df):
    df['Fireplace_Qu'] = df['Fireplace_Qu'].map(
        {'Excellent':5,'Good':4,'Typical':3,'Fair':2,'Poor':1,'No_Fireplace':0})
    return df

In [21]:
X_train_FE_encode=encode_fireplace(X_train_FE_encode)
#test

In [22]:
X_test_FE_encode=encode_fireplace(X_test_FE_encode)

GARAGE

In [23]:
def encode_garage(df):
    df['Garage_Finish'] = df['Garage_Finish'].map(
        {'Fin':3,'RFn':2,'Unf':1,'No_Garage':0})
    df['Garage_Qual'] = df['Garage_Qual'].map(
        {'Excellent':5,'Good':4,'Typical':3,'Fair':2,'Poor':1,'No_Garage':0})
    return df

In [24]:
X_train_FE_encode=encode_garage(X_train_FE_encode)
#test

In [25]:
X_test_FE_encode=encode_garage(X_test_FE_encode)

OVERALL QUALITY AND CONDITION

In [26]:
def encode_overall_quality_condition(df):
    mp = {'Very_Excellent':10,'Excellent':9,'Very_Good':8,'Good':7,'Above_Average':6,
        'Average':10, 'Below_Average':4,'Fair':3,'Poor':2,'Very_Poor':1}
    df['Overall_Qual'] = df['Overall_Qual'].map(mp)
    df['Overall_Cond'] = df['Overall_Cond'].map(mp)
    return df

In [27]:
X_train_FE_encode=encode_overall_quality_condition(X_train_FE_encode)
#test

In [28]:
X_test_FE_encode=encode_overall_quality_condition(X_test_FE_encode)

PAVED DRIVE

In [29]:
X_train_FE_encode['Paved_Drive'].value_counts()

Paved               1977
Dirt_Gravel          175
Partial_Pavement      45
Name: Paved_Drive, dtype: int64

In [30]:
def encode_paved_drive(df):
    df['Paved_Drive'] = df['Paved_Drive'].map(
        {'Paved':2,'Partial_Pavement':1,'Dirt_Gravel':0})
    return df

In [31]:
X_train_FE_encode=encode_paved_drive(X_train_FE_encode)
#test

In [32]:
X_test_FE_encode=encode_paved_drive(X_test_FE_encode)

## One Hot Encoding

In [33]:
X_train_ready = X_train_FE_encode.copy(deep=True) #crea una copia
X_test_ready = X_test_FE_encode.copy(deep=True)

In [34]:
#verifica se X train e X test hanno le stesse colonne
common_cols = X_train_ready.columns.intersection(X_test_ready.columns)
train_not_test = X_train_ready.columns.difference(X_test_ready.columns)

print(common_cols)
print(train_not_test)

Index(['Bedroom_AbvGr', 'Bldg_Type', 'BsmtFin_SF_1', 'BsmtFin_SF_2',
       'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Bsmt_Cond', 'Bsmt_Exposure',
       'Bsmt_Qual', 'Bsmt_Unf_SF', 'Electrical', 'Enclosed_Porch',
       'Exter_Cond', 'Exter_Qual', 'Fireplace_Qu', 'Fireplaces',
       'First_Flr_SF', 'Foundation', 'Garage_Area', 'Garage_Cars',
       'Garage_Finish', 'Garage_Qual', 'Gr_Liv_Area', 'House_Style',
       'Kitchen_Qual', 'Land_Contour', 'Lot_Area', 'Lot_Frontage',
       'Low_Qual_Fin_SF', 'Mas_Vnr_Area', 'Neighborhood', 'Open_Porch_SF',
       'Overall_Cond', 'Overall_Qual', 'Paved_Drive', 'Pool_Area',
       'Roof_Style', 'Sale_Condition', 'Sale_Price', 'Screen_Porch',
       'Second_Flr_SF', 'Street', 'Three_season_porch', 'TotRms_AbvGrd',
       'Total_Bsmt_SF', 'Utilities', 'Wood_Deck_SF', 'id', 'Has_Pool',
       'Total_Bath', 'Has_garage', 'Has_Alley', 'Has_Basement',
       'Has_Fireplace', 'Has_Fence', 'Is_Remodeled', 'House_Age', 'Total_SF',
       'close_to_park', 'c

In [35]:
#N.B: PER SEMPLICITA' supponiamo di essere già a conoscenza di tutte le categorie che possono assumere le variabili categoriali, e che il dataset test non
# comporti l'introduzione di nuove categorie non considerate dal modello
one_hot_cols=['Bldg_Type','Foundation','House_Style','Land_Contour','Neighborhood','Roof_Style','Sale_Condition','Street']
all_data= pd.concat([X_train_ready,X_test_ready],axis=0)
all_data = pd.get_dummies(all_data, columns=one_hot_cols, drop_first=1) 

X_train_ready =  all_data.iloc[:X_train_ready.shape[0]]
X_test_ready = all_data.iloc[-X_test_ready.shape[0]:]


In [88]:
'''
#Label encoding:
Bsmt_Cond ->OK
Bsmt_Qual -> OK
Bsmt_Exposure -> OK
Exter_Cond -> OK
Exter_Qual -> OK
Fireplace_Qu -> OK
Garage_Cond-> delete
Garage_Qual -> OK
Heating_QC-> delete
Kitchen_Qual -> OK
Lot_Shape-> delete
Overall_Cond -> OK
Overall_Qual -> OK

#One Hot encoding:
Alley -> delete
central_air-> delete
Bldg_Type -> OK
BsmtFin_Type_1 -> OK
BsmtFin_Type_2 -> OK
Condition_1 -> delete
Condition_2 -> delete
Electrical -> OK
Exterior_1st-> delete
Exterior_2nd-> delete
Foundation -> OK
Functional-> delete
Garage_Finish -> OK
Garage_Type -> OK
Heating -> OK
House_Style -> OK
Land_Contour -> OK
Land_Slope-> delete
Lot_Config-> delete
MS_SubClass-> delete
MS_Zoning-> delete
Mas_Vnr_Type-> delete
Misc_Feature -> OK
Neighborhood -> OK
Paved_Drive -> OK
Roof_Matl-> delete
Roof_Style -> OK
Sale_Condition -> OK
Sale_Type-> delete
Street -> OK
Utilities -> OK
'''

'\n#Label encoding:\nBsmt_Cond ->OK\nBsmt_Qual -> OK\nBsmt_Exposure -> OK\nExter_Cond -> OK\nExter_Qual -> OK\nFireplace_Qu -> OK\nGarage_Cond-> delete\nGarage_Qual -> OK\nHeating_QC-> delete\nKitchen_Qual -> OK\nLot_Shape-> delete\nOverall_Cond -> OK\nOverall_Qual -> OK\n\n#One Hot encoding:\nAlley -> delete\ncentral_air-> delete\nBldg_Type\nBsmtFin_Type_1\nBsmtFin_Type_2\nCondition_1 -> delete\nCondition_2 -> delete\nElectrical\nExterior_1st-> delete\nExterior_2nd-> delete\nFoundation\nFunctional-> delete\nGarage_Finish\nGarage_Type\nHeating\nHouse_Style\nLand_Contour\nLand_Slope-> delete\nLot_Config-> delete\nMS_SubClass-> delete\nMS_Zoning-> delete\nMas_Vnr_Type-> delete\nMisc_Feature\nNeighborhood\nPaved_Drive\nRoof_Matl-> delete\nRoof_Style\nSale_Condition\nSale_Type-> delete\nStreet\nUtilities\n'

# OUTLIERS
utilizziamo una random forest per individuare gli outliers del dataset di train, per poi provare ad allenare due modelli(uno su dataset con outliers e uno senza) e vedere come si comportano su dataset di test

In [37]:
identify_columns_with_na(X_train_ready)


,column,na_count


In [41]:
iso = IsolationForest()
param_grid = {'n_estimators': [1000, 2000], #numero di alberi nella foresta
              'max_samples': ['auto'], #numero di campioni prelevati per addestrare gli alberi
              'contamination': ['auto'],  #proporzione attesa di outliers nel dataset <-- Molto sensibile
              'bootstrap' : [True],
              'max_features': [1.0, 0.5, 0.3]
              } #numero di feature su cui sono allenati gli alberi

grid_search = model_selection.GridSearchCV(iso, 
                                           param_grid,
                                           scoring="neg_root_mean_squared_error", #MSE MEAN SQUARE ERROR
                                           verbose=3,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)

X_temp=X_train_ready.drop(["id","Sale_Price"], axis=1) #rimuovo id

grid_search.fit(X_temp.values, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid

best_model = grid_search.fit(X_temp.values, y_train)
print('Optimum parameters', best_model.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Optimum parameters {'bootstrap': True, 'contamination': 'auto', 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 2000}


In [42]:
'''Optimum parameters {'bootstrap': True, 'contamination': 'auto', 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 1000}'''

iso = IsolationForest(n_estimators=1000, contamination='auto', max_samples='auto', bootstrap='True', max_features=1.0) #max_features='default' non viene fatto subset delle feature, max_features è il numero di features del dataset
'''
#se volessi dare più peso ad una feature allora dovrei duplicarla nel dataset
df1=df.copy()
weight_feature = 10
for i in range(weight_feature):
    df1["duplicated_" + str(i)] = df1["feature_to_duplicate"]
'''
iso.fit(X_temp.values, y_train) #l'isolation forest viene allenata su tutte le feature del dataset

IsolationForest(bootstrap='True', n_estimators=1000)

In [43]:
X_temp=X_temp.assign(anomaly=iso.predict(X_temp.values)) #crea nuova colonna "anomaly" e la aggancia, -1 se l'istanza è considerata outlier, 1 altrimenti
#X_train_ready['anomaly'] = iso.predict(X_train_ready)

In [44]:
anomaly=X_temp.loc[X_temp['anomaly']==-1] #crea un dataframe con le istanze identificate outliers
anomaly_index=list(anomaly.index) #salva una lista degli index degli outliers
print(anomaly.shape) 

(28, 115)


In [18]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.express.colors import sample_colorscale
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay

In [46]:
X_temp['anomaly'] = X_temp['anomaly'].astype(str)
fig=px.scatter(X_temp, x="Gr_Liv_Area", y=y_train, color="anomaly",width=1000, height=600)
fig.show()
X_temp['anomaly']=X_temp['anomaly'].astype(float)

In [47]:
print(f'contamination used: {anomaly.shape[0]/2197}')

contamination used: 0.012744651797906235


In [48]:
anomaly.to_csv('dataset/outliers.csv') #salvo gli outliers in un csv

In [30]:
outliers = pd.read_csv('dataset/outliers.csv', index_col=0)

In [51]:
outliers=X_train_ready.loc[outliers.index]

In [69]:
outliers_index = outliers["id"].tolist()
X_train_ready["anomaly"] = np.where(X_train_ready["id"].isin(outliers_index), -1, 1).astype(str)
pca=PCA(n_components=3)
components = pca.fit_transform(X_train_ready)
total_var = pca.explained_variance_ratio_.sum()*100
fig = px.scatter_3d(components, x=0, y=1,z=2, hover_data=[X_train_ready.index], color=X_train_ready["anomaly"], title=f'Total Explained Variance: {total_var:.2f}%', labels={'0': 'PC1', '1': 'PC2', '2':'PC3'})
fig.show()

In [49]:
X_train_ready.to_csv('dataset/train_data_FE_encoded_With_outliers.csv', index=False) #esporto un csv con gli outliers

In [50]:
X_train_ready=X_train_ready.drop(anomaly_index, axis=0) #rimuovo la colonna che segna le anomalie

In [51]:
y_train.to_csv("dataset/y_train_With_outliers.csv", index=False)

In [52]:
y_train = y_train.drop(y_train.index[anomaly_index]) #rimuovo anche da y_train i SalePrice relativo alle anomalie

In [53]:
fig=px.scatter(X_train_ready, x="Gr_Liv_Area", y="Sale_Price" ,width=1000, height=600)
fig.show()

In [54]:
X_train_ready.to_csv('dataset/train_data_FE_encoded_No_outliers.csv', index=False) #esporto un csv senza outliers

In [55]:
X_test_ready.to_csv('dataset/test_data_FE_encoded.csv', index=False) #esporto in csv il test con variabili dopo encoding

In [56]:
y_train.to_csv('dataset/y_train.csv', index=False) #esporto in csv y_train senza i SalePrice degli outliers

In [94]:
def save_session() :
    dill.dump_session('notebook_env.db')

def load_session():
    dill.load_session('notebook_env.db')

In [3]:
#load_session()

In [95]:
#save_session()